📘 Employee Churn Simulation — Full Walkthrough (Markdown Edition)
Author: Roy Phelps
Project: HR / People Analytics — Churn Analysis Simulation
Companion Notebook: 01_data_ingest.ipynb
This file: A full narrative walkthrough of every step in the churn pipeline, written for clarity, learning, and interview discussions (CACI-ready).

📚 0. Overview
This walkthrough explains each step performed in the employee churn analysis pipeline:
What each code block does
Why it matters analytically
How to confidently discuss it during your CACI People Analytics Specialist call
How this simulates real Workday/PeopleSoft/Workforce HR data processing
This markdown reads like a mini data engineering + people analytics case study.

🏁 1. Project Context & Goals
We are simulating a real People Analytics workflow using HR data:
Employee roster table
Status history table (hire, active changes, terminations)
Department dimension
API metadata containing a dataset extraction timestamp
Our goals:
Build a master employee dataset with:
Current status
Termination date (inferred or explicit)
Tenure (days, years)
Department info
Compute monthly churn metrics:
Hires
Terminations
Month-end headcount
Churn rate = terminations ÷ headcount
Prepare material you can discuss fluently with CACI as an example project.

📥 2. Data Ingest – Reading the HR Tables
Files loaded:
File	Purpose
workforce_employees.csv	One row per employee (demographics + hire/termination dates)
workforce_status_history.csv	Timeline of events (Hire, Active, Terminated, etc.)
workforce_departments.csv	Dimension table mapping department_id → department_name
workforce_employees_api.json	Mock API metadata, including extracted_at timestamp
Why this matters
Every HR system (Workday, Oracle, ADP) has a similar structure.
You are replicating what a People Analytics Specialist actually ingests daily.
The JSON “extracted_at” timestamp becomes your snapshot date, which is crucial.
Talking point for CACI
“I ingested HR snapshot files — employee roster, status history, departments, plus metadata — and used the API timestamp as the reporting ‘as-of’ date for all calculations.”

🔍 3. Data Inspection – Sanity Checking
Before transforming anything, we check:
Row counts
Column names
Data types
Sample rows
Why this step is important:
Confirms that strings like "2019-01-10" need conversion to real dates
Verifies whether columns are named differently than expected
Catches mismatches early (e.g., effective_date vs status_date)
Talking point:
“I verified the schema of each table, checked the date fields, and confirmed the shape of the data before modeling it.”

🗂️ 4. Defining the Status Date Column
The status history table uses:
effective_date
as the date each change took effect.
We normalize it:
Convert to a real datetime
Force it to UTC
Drop timezone afterwards
Why:
HR data often mixes timezone formats
Pandas date arithmetic requires consistent types
Prevents errors during tenure calculations
Talking point:
“I standardized all date columns to timezone-naive datetimes to ensure consistent sorting and arithmetic.”

🔎 5. Extracting Latest Status Per Employee
The status history table may look like:
employee_id	effective_date	status
1002	2020-05-01	Active
1002	2023-02-15	Terminated
We:
Sort all records by employee_id → effective_date
Take the last record for each employee
Rename:
status → current_status
effective_date → status_as_of
Why this matters:
HR status history is long format
For reporting, we need wide format: one row per employee
This mirrors real-world ETL pipelines in HR analytics
Talking point:
“I collapsed the employee status history into a single row per employee representing their current status as of the data extract.”

🏢 6. Adding Department Names
We join employees → departments using department_id.
Why:
Department codes like ENG mean nothing to leadership
Department names like Engineering or Finance do
Talking point:
“I enriched the employee roster with department names to enable churn reporting by org group.”

⏱️ 7. Parsing the Snapshot Date from API Metadata
The JSON file includes:
"extracted_at": "2025-11-16T19:12:55.944666Z"
This becomes the official “as-of” timestamp for:
Tenure calculations
Month-end headcount logic
Any reporting cutoff
We parse it into a clean datetime.
Talking point:
“I treated the API’s extracted_at timestamp as the official as-of date for all HR metrics, just like a real HRIS export.”

🧩 8. Building the Master Employee Table (master_employee_df)
This is your gold table.
It includes:
Employee demographic details
Department name
Current status + effective date
Hire date
Termination date (explicit or inferred)
Tenure
Status timeline summary
How it's built:
Merge employees + department_name
Merge with latest_status
Convert hire/termination to datetime
If termination is missing, substitute snapshot date
Compute:
tenure_days = end_date - hire_date
tenure_years = tenure_days / 365.25
This creates the dataset recruiters love to see.
Talking point:
“I built a master employee table with current status, department, hire/termination dates, and computed tenure for all employees. This table becomes the foundation for churn calculations and BI dashboards.”

📅 9. Deriving Monthly Hires & Terminations
We turn status history into month-level events:
Convert each effective date into a month period
Classify each row as:
hire
termination
other
Count unique employees per event per month
Why:
Raw rows → easy-to-understand leadership metrics
Monthly activity is the base for churn calculations
Talking point:
“I derived monthly hire and termination counts by grouping status events by month and event type.”

🧮 10. Calculating Month-End Headcount
This step simulates real HR analytics logic:
For each month from the earliest hire → snapshot date:
Employees are active on month-end if:
Hire date ≤ month_end
And termination date is empty OR > month_end
Why:
HR systems do not give “headcount per month” automatically
You have to compute it based on employment windows
This is essential for accurate churn rate math
Talking point:
“I computed month-end headcount by determining which employees were active at each month’s close, based on their hire and termination dates.”

📉 11. Monthly Churn Rate
Final step:
churn_rate = terminations / headcount_end
We merge:
Monthly hires/terminations
Month-end headcount
And produce a complete time series of:
Hires
Terminations
Headcount
Churn rate
Talking point:
“I combined monthly hires, terminations, and headcount to calculate monthly churn rates, producing insights similar to what HR leadership expects from a workforce analytics dashboard.”

🎯 12. Final Output — What This Pipeline Produces
You have created:
✔️ A master employee table
Ready for BI use or for feeding a predictive churn model
✔️ A monthly events table
Hires, terminations, other events
✔️ A month-end headcount table
Accurate active counts
✔️ A monthly churn table
The key HR metric